##토큰화와 벡터화 같은 기본적인 NLP 전처리를 마친 후 pycaret 적용



1.   BOW-TF (Countvectorizer)
2.   BOW-TFIDF (TFIDFvectorizer) ☑





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip show packaging

Name: packaging
Version: 21.3
Summary: Core utilities for Python packages
Home-page: https://github.com/pypa/packaging
Author: Donald Stufft and individual contributors
Author-email: donald@stufft.io
License: BSD-2-Clause or Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: pyparsing
Required-by: arviz, astropy, bokeh, build, dask, db-dtypes, distributed, fastai, fastcore, geopandas, google-cloud-bigquery, h5netcdf, holoviews, jupyter-server, matplotlib, nbconvert, pooch, pytest, scikit-image, spacy, Sphinx, statsmodels, tables, tensorflow, thinc, xarray


In [3]:
pip install "packaging<=21.3"

In [4]:
from packaging.version import Version

version_string = "21.3"
version = Version(version_string)

print(version)

21.3




*   하단의 코드에서 패키징 버전이 23.1이 아닌 **21.3 이하 버전**일 때 pycaret import 가능

In [5]:
import packaging

print(packaging.__version__)

21.3


pycaret 설치 오류 관련 링크 (버전 문제) <br/>
https://coding-potato.tistory.com/7

pycaret 동작 방법 <br/>
https://www.notion.so/pycaret-836baab165694d519ad90fd8c1dc6713?pvs=4

In [6]:
#pycaret 돌릴 때 필요한 코드, 단 처음 돌린 후 런타임 재시작하고 다시 돌려야 돌아가는 단점 존재
!pip install -U pypandoc
!pip install  pycaret[full]
#!pip install markupsafe==2.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.4/484.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [7]:
from pycaret.classification import *

In [8]:
import re
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
import warnings
warnings.filterwarnings(action='ignore')

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [11]:
train_r = '/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/train.csv'
test_r= '/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/test.csv'
temp_r='/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/0702_0.5_train.csv'
tempT_r='/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/0702_0.5_test.csv'

In [12]:
train = pd.read_csv(train_r)
test = pd.read_csv(test_r)
new = pd.read_csv(temp_r)
newT=pd.read_csv(tempT_r)

In [13]:
new

,first_party_ner_ORG,first_party_ner_PERSON,first_party_ner_other,first_party_ner_others,second_party_ner_ORG,second_party_ner_PERSON,second_party_ner_other,second_party_ner_others,issued_area_criminal,issued_area_others,...,well,whether,within,without,woman,work,would,writ,year,york
0,0,1,0,0,0,1,0,0,0,1,...,0.0,0.153772,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,0,1,0,0,0,1,0,0,1,0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.284502,0.091068,0.0
2,0,1,0,0,0,0,0,0,1,0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
3,0,0,0,1,0,0,0,1,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
4,0,1,0,0,0,0,0,0,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.116072,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,1,0,0,0,1,0,0,0,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.094845,0.000000,0.195227,0.0
2474,0,1,0,0,1,0,0,0,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.087753,0.000000,0.000000,0.0
2475,1,0,0,0,0,0,0,0,1,0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
2476,1,0,0,0,0,0,0,1,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [14]:
newT

,first_party_ner_ORG,first_party_ner_PERSON,first_party_ner_other,first_party_ner_others,second_party_ner_ORG,second_party_ner_PERSON,second_party_ner_other,second_party_ner_others,issued_area_criminal,issued_area_others,...,well,whether,within,without,woman,work,would,writ,year,york
0,0,0,0,1,0,0,0,0,0,1,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,0,1,0,0,1,0,0,0,1,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,1,0,0,0,1,0,0,0,1,0,...,0.0,0.0,0.0,0.146081,0.0,0.0,0.0,0.0,0.0,0.000000
3,0,1,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,0,1,0,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,1,0,0,0,1,0,0,0,1,0,...,0.0,0.0,0.0,0.124133,0.0,0.0,0.0,0.0,0.0,0.167817
1236,0,0,0,1,1,0,0,0,1,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1237,0,1,0,0,0,1,0,0,0,1,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1238,1,0,0,0,0,1,0,0,0,1,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [ ]:
#new.drop(columns=['Unnamed: 0','first_party','second_party'],inplace=True)
#newT.drop(columns=['Unnamed: 0','first_party','second_party'],inplace=True)
#new

KeyError: ignored

In [15]:
newT['ID']=test['ID']
newT

,first_party_ner_ORG,first_party_ner_PERSON,first_party_ner_other,first_party_ner_others,second_party_ner_ORG,second_party_ner_PERSON,second_party_ner_other,second_party_ner_others,issued_area_criminal,issued_area_others,...,whether,within,without,woman,work,would,writ,year,york,ID
0,0,0,0,1,0,0,0,0,0,1,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,TEST_0000
1,0,1,0,0,1,0,0,0,1,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,TEST_0001
2,1,0,0,0,1,0,0,0,1,0,...,0.0,0.0,0.146081,0.0,0.0,0.0,0.0,0.0,0.000000,TEST_0002
3,0,1,0,0,0,0,0,0,0,1,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,TEST_0003
4,0,1,0,0,0,0,0,1,0,1,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,TEST_0004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,1,0,0,0,1,0,0,0,1,0,...,0.0,0.0,0.124133,0.0,0.0,0.0,0.0,0.0,0.167817,TEST_1235
1236,0,0,0,1,1,0,0,0,1,0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,TEST_1236
1237,0,1,0,0,0,1,0,0,0,1,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,TEST_1237
1238,1,0,0,0,0,1,0,0,0,1,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,TEST_1238


In [16]:
new=pd.merge(new,train)
newT=pd.merge(newT,test)

In [17]:
new

,first_party_ner_ORG,first_party_ner_PERSON,first_party_ner_other,first_party_ner_others,second_party_ner_ORG,second_party_ner_PERSON,second_party_ner_other,second_party_ner_others,issued_area_criminal,issued_area_others,...,without,woman,work,would,writ,year,york,first_party,second_party,facts
0,0,1,0,0,0,1,0,0,0,1,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ..."
1,0,1,0,0,0,1,0,0,1,0,...,0.0,0.0,0.0,0.000000,0.284502,0.091068,0.0,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...
2,0,1,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...
3,0,0,0,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Linkletter,Walker,Victor Linkletter was convicted in state court...
4,0,1,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.116072,0.000000,0.000000,0.0,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,1,0,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.094845,0.000000,0.195227,0.0,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...
2474,0,1,0,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.087753,0.000000,0.000000,0.0,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ..."
2475,1,0,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Peguero,United States,"In 1992, the District Court sentenced Manuel D..."
2476,1,0,0,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per..."


In [18]:
new.set_index('ID', drop = True, inplace=True)
newT.set_index('ID', drop = True, inplace=True)

In [19]:
new.drop(columns=['facts'],inplace=True)
newT.drop(columns=['facts'],inplace=True)

In [ ]:
new

,first_party_ner_ORG,first_party_ner_PERSON,first_party_ner_other,first_party_ner_others,second_party_ner_ORG,second_party_ner_PERSON,second_party_ner_other,second_party_ner_others,issued_area_criminal,issued_area_others,...,well,whether,within,without,woman,work,would,writ,year,york
ID,,,,,,,,,,,,,,,,,,,,,
TRAIN_0000,0,1,0,0,0,1,0,0,0,1,...,0.0,0.153772,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
TRAIN_0001,0,1,0,0,0,1,0,0,1,0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.284502,0.091068,0.0
TRAIN_0002,0,1,0,0,0,0,0,0,1,0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
TRAIN_0003,0,0,0,1,0,0,0,1,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
TRAIN_0004,0,1,0,0,0,0,0,0,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.116072,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_2473,1,0,0,0,1,0,0,0,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.094845,0.000000,0.195227,0.0
TRAIN_2474,0,1,0,0,1,0,0,0,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.087753,0.000000,0.000000,0.0
TRAIN_2475,1,0,0,0,0,0,0,0,1,0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [20]:
newT

,first_party_ner_ORG,first_party_ner_PERSON,first_party_ner_other,first_party_ner_others,second_party_ner_ORG,second_party_ner_PERSON,second_party_ner_other,second_party_ner_others,issued_area_criminal,issued_area_others,...,within,without,woman,work,would,writ,year,york,first_party,second_party
ID,,,,,,,,,,,,,,,,,,,,,
TEST_0000,0,0,0,1,0,0,0,0,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,Salerno,United States
TEST_0001,0,1,0,0,1,0,0,0,1,0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc."
TEST_0002,1,0,0,0,1,0,0,0,1,0,...,0.0,0.146081,0.0,0.0,0.0,0.0,0.0,0.000000,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al."
TEST_0003,0,1,0,0,0,0,0,0,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,Harold Kaufman,United States
TEST_0004,0,1,0,0,0,0,0,1,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,Berger,Hanlon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TEST_1235,1,0,0,0,1,0,0,0,1,0,...,0.0,0.124133,0.0,0.0,0.0,0.0,0.0,0.167817,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A..."
TEST_1236,0,0,0,1,1,0,0,0,1,0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,Whitman,"American Trucking Associations, Inc."
TEST_1237,0,1,0,0,0,1,0,0,0,1,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,Linda A. Matteo and John J. Madigan,William G. Barr


In [21]:
#fix_imbalance=True를 통해 SMOTE를 적용하여 클래스가 적은 데이터의 표본을 임의 값으로 변형하여 데이터 추가 적용
exp=setup(data=new, target='first_party_winner', session_id=4454, fix_imbalance=True, normalize=True)

,Description,Value
0,Session id,4454
1,Target,first_party_winner
2,Target type,Binary
3,Original data shape,"(2478, 418)"
4,Transformed data shape,"(3052, 418)"
5,Transformed train set shape,"(2308, 418)"
6,Transformed test set shape,"(744, 418)"
7,Numeric features,415
8,Categorical features,2
9,Preprocess,True


In [22]:
 best_15 = compare_models(sort='Accuracy', n_select=15)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
qda,Quadratic Discriminant Analysis,0.6655,0.5004,1.0000,0.6655,0.7992,0.0000,0.0000,1.4470
et,Extra Trees Classifier,0.6580,0.5635,0.9333,0.6760,0.7841,0.0539,0.0769,1.7660
gbc,Gradient Boosting Classifier,0.6569,0.5524,0.9107,0.6817,0.7791,0.0730,0.0879,4.7840
catboost,CatBoost Classifier,0.6499,0.5493,0.9072,0.6775,0.7750,0.0520,0.0595,79.2560
xgboost,Extreme Gradient Boosting,0.6464,0.5268,0.9341,0.6674,0.7785,0.0104,0.0171,2.8740
lightgbm,Light Gradient Boosting Machine,0.6378,0.5268,0.8994,0.6698,0.7676,0.0197,0.0238,2.3110
rf,Random Forest Classifier,0.6314,0.5717,0.8291,0.6838,0.7487,0.0757,0.0832,2.4280
ridge,Ridge Classifier,0.6234,0.0000,0.8769,0.6644,0.7559,-0.0047,-0.0055,0.5550
lda,Linear Discriminant Analysis,0.6234,0.5062,0.8769,0.6644,0.7559,-0.0047,-0.0055,1.1440
lr,Logistic Regression,0.6194,0.5247,0.8319,0.6734,0.7442,0.0317,0.0338,1.4580


Processing:   0%|          | 0/83 [00:00<?, ?it/s]


### ✔ Gradient Boosting Classifier
*   **Predict 0 Result** </br>
train : 684 </br>
test  : 181 </br> </br>

acuuracy : 0.6534

In [43]:
%time tuned_model = tune_model(best_15[2],choose_better=True, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6322,0.5432,0.8879,0.6688,0.7630,0.0103,0.0127
1,0.6552,0.5821,0.9569,0.6687,0.7872,0.0110,0.0194
2,0.6494,0.5548,0.9655,0.6627,0.7860,-0.0223,-0.0487
3,0.6897,0.6132,0.9138,0.7067,0.7970,0.1818,0.2121
4,0.6243,0.5156,0.8609,0.6689,0.7529,0.0187,0.0215
5,0.6416,0.4842,0.9217,0.6667,0.7737,0.0098,0.0138
6,0.6763,0.5711,0.8870,0.7034,0.7846,0.1670,0.1866
7,0.6474,0.5187,0.8609,0.6875,0.7645,0.0970,0.1074
8,0.6532,0.5949,0.8783,0.6871,0.7710,0.0986,0.1125


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
CPU times: user 11.9 s, sys: 5.96 s, total: 17.8 s
Wall time: 2min 17s


In [59]:
tuned_model

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=4454, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [44]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.8939,0.9487,0.9642,0.8863,0.9236,0.7508,0.7580


In [45]:
y_pred.value_counts()

1    1794
0     684
Name: prediction_label, dtype: int64

In [46]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [47]:
y_pred.value_counts()

1    1059
0     181
Name: prediction_label, dtype: int64

In [48]:
y_pred=y_pred.to_frame()

In [49]:
y_pred

,prediction_label
ID,
TEST_0000,0
TEST_0001,1
TEST_0002,1
TEST_0003,0
TEST_0004,1
...,...
TEST_1235,1
TEST_1236,1
TEST_1237,1


In [50]:
index_list = list(range(1240))

In [51]:
index_list=[index_list]

In [52]:
index_list

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  15

In [53]:
y_pred = y_pred.set_index(keys=index_list, inplace=False, drop=False)

In [54]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/sample_submission.csv')

In [55]:
submit

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1235,TEST_1235,0
1236,TEST_1236,0
1237,TEST_1237,0
1238,TEST_1238,0


In [56]:
submit['first_party_winner'] = y_pred['prediction_label']

In [57]:
submit

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,1
2,TEST_0002,1
3,TEST_0003,0
4,TEST_0004,1
...,...,...
1235,TEST_1235,1
1236,TEST_1236,1
1237,TEST_1237,1
1238,TEST_1238,1


In [58]:
submit.to_csv('./pycaretGBC.csv', index=False)
print('Done')

Done



### ✔ Light Gradient Boosting Machine
*   **Predict 0 Result** </br>
train : 644 </br>
test  : 118 </br> </br>

acuuracy : 0.6569

In [33]:
%time tuned_model = tune_model(best_15[5],choose_better=True, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6437,0.5434,0.9138,0.6709,0.7737,0.0211,0.0281
1,0.6609,0.5990,0.9483,0.6748,0.7885,0.0432,0.0668
2,0.6494,0.5152,0.9655,0.6627,0.7860,-0.0223,-0.0487
3,0.6897,0.6064,0.9569,0.6938,0.8043,0.1383,0.1942
4,0.6532,0.5213,0.9043,0.6797,0.7761,0.0711,0.0879
5,0.6647,0.4876,0.9391,0.6792,0.7883,0.0737,0.1035
6,0.6705,0.5178,0.9043,0.6933,0.7849,0.1308,0.1547
7,0.6358,0.5117,0.9304,0.6605,0.7726,-0.0223,-0.0345
8,0.6474,0.5526,0.8435,0.6929,0.7608,0.1143,0.1227


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


CPU times: user 12.4 s, sys: 4.26 s, total: 16.6 s
Wall time: 2min 53s


In [34]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8939,0.9540,0.9763,0.8779,0.9245,0.7476,0.7597


In [35]:
y_pred.value_counts()

1    1834
0     644
Name: prediction_label, dtype: int64

In [36]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [37]:
y_pred.value_counts()

1    1122
0     118
Name: prediction_label, dtype: int64

In [ ]:
y_pred

,prediction_label
0,0
1,0
2,1
3,1
4,1
...,...
1235,1
1236,0
1237,1
1238,1


In [ ]:
tuned_model


### ✔ Random Forest
*   **Predict '0' Result** </br>
train : 704 </br>
test  : 198 </br></br>

accuracy : 0.5999

In [38]:
%time tuned_model = tune_model(best_15[6],choose_better=True, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3793,0.5216,0.0948,0.7857,0.1692,0.0299,0.0747
1,0.3678,0.5172,0.0690,0.8000,0.1270,0.0237,0.0698
2,0.6609,0.5431,0.8966,0.6887,0.7790,0.1015,0.1200
3,0.6954,0.5862,0.9138,0.7114,0.8000,0.2010,0.2317
4,0.6301,0.5209,0.8522,0.6759,0.7538,0.0480,0.0536
5,0.6301,0.5166,0.8609,0.6735,0.7557,0.0386,0.0440
6,0.6879,0.5986,0.8696,0.7194,0.7874,0.2197,0.2342
7,0.6821,0.5729,0.9043,0.7027,0.7909,0.1696,0.1957
8,0.6474,0.5468,0.8522,0.6901,0.7626,0.1058,0.1152


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
CPU times: user 12.1 s, sys: 5.43 s, total: 17.5 s
Wall time: 5min 31s


In [39]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8947,0.9606,0.9588,0.8912,0.9238,0.7542,0.7595


In [40]:
y_pred.value_counts()

1    1774
0     704
Name: prediction_label, dtype: int64

In [41]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [42]:
y_pred.value_counts()

1    1042
0     198
Name: prediction_label, dtype: int64


### ✔ Extra Trees Classifier
*   **Predict 0 Result** </br>
train : 639 </br>
test  : 130 </br></br>

accuracy : 0.5553


In [ ]:
tuned_model = tune_model(best_15[2])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6322,0.6033,0.8276,0.6857,0.7500,0.0769,0.0820
1,0.6437,0.5617,0.8448,0.6901,0.7597,0.0971,0.1049
2,0.6207,0.5859,0.8362,0.6736,0.7462,0.0294,0.0323
3,0.6494,0.5830,0.8448,0.6950,0.7626,0.1159,0.1244
4,0.5780,0.4642,0.7913,0.6500,0.7137,-0.0599,-0.0643
5,0.6474,0.6360,0.9304,0.6688,0.7782,0.0206,0.0298
6,0.6301,0.5657,0.8174,0.6861,0.7460,0.0839,0.0884
7,0.6705,0.6154,0.8870,0.6986,0.7816,0.1479,0.1669
8,0.6416,0.5631,0.8522,0.6853,0.7597,0.0867,0.0952


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8947,0.9646,0.9612,0.8895,0.9239,0.7536,0.7596


In [ ]:
y_pred.value_counts()

1    1782
0     696
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

1    1057
0     183
Name: prediction_label, dtype: int64


### ✔ Naive Bayes
*   **Predict 0 Result** </br>
train : 696 </br>
test  : 556 </br></br>

accuracy :  0.5325


In [ ]:
tuned_model = tune_model(best_15[2])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5702,0.5182,0.4464,0.5814,0.5051,0.1366,0.1404
1,0.4825,0.4843,0.4464,0.4717,0.4587,-0.0364,-0.0364
2,0.6053,0.5973,0.5714,0.6038,0.5872,0.2096,0.2098
3,0.4035,0.4073,0.3036,0.3696,0.3333,-0.1970,-0.2002
4,0.5000,0.4757,0.5179,0.4915,0.5043,0.0006,0.0006
5,0.5965,0.5713,0.5179,0.6042,0.5577,0.1907,0.1927
6,0.5439,0.5533,0.5357,0.5357,0.5357,0.0874,0.0874
7,0.5000,0.5148,0.4286,0.4898,0.4571,-0.0025,-0.0025
8,0.5965,0.6259,0.4464,0.6250,0.5208,0.1887,0.1967


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.6501,0.7075,0.5788,0.6652,0.6190,0.2984,0.3008


In [ ]:
y_pred.value_counts()

0    933
1    696
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

0    684
1    556
Name: prediction_label, dtype: int64



---




### ✔ XGBoost
*   **Predict 0 Result** </br>
train : 625 </br>
test  : 103 </br>

In [ ]:
tuned_model = tune_model(best_15[3])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6609,0.5312,0.9741,0.6686,0.7930,0.0112,0.0243
1,0.6609,0.5584,0.9655,0.6707,0.7915,0.0221,0.0414
2,0.6494,0.5108,0.9655,0.6627,0.7860,-0.0223,-0.0487
3,0.6839,0.5813,0.9914,0.6805,0.8070,0.0782,0.1703
4,0.6590,0.4667,0.9565,0.6707,0.7885,0.0322,0.0542
5,0.6532,0.5039,0.9478,0.6687,0.7842,0.0211,0.0340
6,0.6879,0.5596,0.9217,0.7020,0.7970,0.1724,0.2067
7,0.6416,0.5094,0.9304,0.6646,0.7754,-0.0007,-0.0011
8,0.6532,0.6121,0.8870,0.6846,0.7727,0.0896,0.1046


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8951,0.9450,0.9830,0.8748,0.9258,0.7490,0.7640


In [ ]:
y_pred.value_counts()

1    1853
0     625
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

1    1137
0     103
Name: prediction_label, dtype: int64


### ✔ CatBoost
*   **Predict 0 Result** </br>
train : 627 </br>
test  : 97 </br>

*이것만 돌아가는데 한 시간 20분 걸렸어요..

In [ ]:
tuned_model = tune_model(best_15[4])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6379,0.5273,0.9052,0.6688,0.7692,0.0105,0.0137
1,0.6609,0.5477,0.9655,0.6707,0.7915,0.0221,0.0414
2,0.6552,0.5425,0.9655,0.6667,0.7887,0.0000,0.0000
3,0.6667,0.5933,0.9052,0.6908,0.7836,0.1122,0.1345
4,0.6243,0.5067,0.8783,0.6645,0.7566,-0.0012,-0.0015
5,0.6590,0.5370,0.9478,0.6728,0.7870,0.0426,0.0658
6,0.6821,0.5732,0.8957,0.7055,0.7893,0.1778,0.2007
7,0.6301,0.5090,0.9217,0.6584,0.7681,-0.0330,-0.0493
8,0.6705,0.6378,0.9565,0.6790,0.7942,0.0750,0.1160


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,CatBoost Classifier,0.8902,0.9527,0.9788,0.8720,0.9223,0.7376,0.7520


In [ ]:
y_pred.value_counts()

1    1851
0     627
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

1    1143
0      97
Name: prediction_label, dtype: int64

In [ ]:
tuned_model

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/sample_submission.csv')
submit['first_party_winner'] = y_pred
submit.to_csv('./proces_tf_catboost.csv', index=False)
print('Done')

Done



### ✔ LGBM
*   **Predict 0 Result** </br>
train : 689 </br>
test  : 181 </br>

In [ ]:
tuned_model = tune_model(best_15[5])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6149,0.5352,0.8276,0.6713,0.7413,0.0195,0.0212
1,0.6667,0.5553,0.8793,0.6986,0.7786,0.1386,0.1548
2,0.6552,0.5465,0.9655,0.6667,0.7887,0.0000,0.0000
3,0.6839,0.6058,0.9052,0.7047,0.7925,0.1709,0.1970
4,0.6301,0.5002,0.8696,0.6711,0.7576,0.0289,0.0338
5,0.6647,0.4895,0.9478,0.6770,0.7899,0.0638,0.0953
6,0.6821,0.5810,0.8957,0.7055,0.7893,0.1778,0.2007
7,0.6647,0.5497,0.8957,0.6913,0.7803,0.1200,0.1401
8,0.6474,0.6090,0.8435,0.6929,0.7608,0.1143,0.1227


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8894,0.9494,0.9594,0.8843,0.9203,0.7408,0.7474


In [ ]:
y_pred.value_counts()

1    1789
0     689
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

1    1059
0     181
Name: prediction_label, dtype: int64


### ✔ Naive Bayes
*   **Predict 0 Result** </br>
train : 1126 </br>
test  : 540 </br>

*이건 너무 많지 않나..?

In [ ]:
tuned_model = tune_model(best_15[13])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5747,0.5477,0.6466,0.6944,0.6696,0.0750,0.0754
1,0.5230,0.5087,0.5603,0.6701,0.6103,0.0080,0.0082
2,0.5230,0.5130,0.5690,0.6667,0.6140,0.0000,0.0000
3,0.5747,0.5539,0.5603,0.7386,0.6373,0.1462,0.1544
4,0.5723,0.5489,0.6435,0.6916,0.6667,0.0720,0.0724
5,0.5376,0.4960,0.5739,0.6804,0.6226,0.0366,0.0375
6,0.5145,0.4828,0.5565,0.6598,0.6038,-0.0116,-0.0118
7,0.5434,0.5231,0.5652,0.6915,0.6220,0.0598,0.0618
8,0.5202,0.5222,0.5652,0.6633,0.6103,-0.0035,-0.0036


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.6533,0.7077,0.6495,0.7922,0.7138,0.2851,0.2943


In [ ]:
y_pred.value_counts()

1    1352
0    1126
Name: prediction_label, dtype: int64

In [ ]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [ ]:
y_pred.value_counts()

1    700
0    540
Name: prediction_label, dtype: int64

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/sample_submission.csv')

In [ ]:
submit['first_party_winner'] = y_pred

In [ ]:
submit.to_csv('./proces_tf_naive.csv', index=False)
print('Done')

Done


In [ ]:
#blended_l = blend_models(estimator_list = best_5, fold = 5, optimize = 'Accuracy')
#pred_holdout = predict_model(blended_l)
#final_model_l = finalize_model(blended_l)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6657,0.5494,1.0000,0.6657,0.7993,0.0000,0.0000
1,0.6657,0.5842,1.0000,0.6657,0.7993,0.0000,0.0000
2,0.6657,0.5679,1.0000,0.6657,0.7993,0.0000,0.0000
3,0.6657,0.5860,1.0000,0.6657,0.7993,0.0000,0.0000
4,0.6647,0.6283,1.0000,0.6647,0.7986,0.0000,0.0000
Mean,0.6655,0.5832,1.0000,0.6655,0.7992,0.0000,0.0000
Std,0.0004,0.0262,0.0000,0.0004,0.0003,0.0000,0.0000


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.6653,0.5416,1.0000,0.6653,0.7990,0.0000,0.0000
